# Segment Classifier

This notebook reports the methodology followed to build a machine learning-based classifier to automatically identify sections in job description documents. The model takes a sentence contained in a job description as input and produces as output the section that the sentence belongs to.

# Section 0. Preliminaries

## Load libraries

In [9]:
# update accordingly
run_on_google_colab = True
project_dir = ''

if run_on_google_colab:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [10]:
import sys
sys.path.append(project_dir)

In [54]:
import json
import joblib
import numpy as np
import os
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics import classification_report, f1_score, make_scorer
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from text_processor import *
from utils import *

## Initialize constants

In [46]:
OVERWRITE_DOC2VEC = False
OVERWRITE_TRAINING = False
random_state = np.random.RandomState(1234)  # for reproducibility

## Load data

In [13]:
data_dir = f'{project_dir}/data'
data_fn = 'jobs_training.csv'
models_dir = f'{project_dir}/models'
output_dir = f'{project_dir}/outputs'

In [14]:
job_training_df = pd.read_csv(os.path.join(data_dir, data_fn))

## Preview data

Visualize the size of the data togethet with a small sample

In [15]:
print(f'dataset size: {job_training_df.shape[0]}x{job_training_df.shape[1]}')
print('-'*10)
job_training_df.head()

dataset size: 3885x4
----------


,job_id,segment_index,segment,section_label
0,05b865e93e8e46579075562865973d3b,0,Abbott is a global healthcare leader that help...,About Company
1,05b865e93e8e46579075562865973d3b,94,Our portfolio of life-changing technologies sp...,About Company
2,05b865e93e8e46579075562865973d3b,287,"Our 109,000 colleagues\nserve people in more t...",About Company
3,05b865e93e8e46579075562865973d3b,353,"**Tissue Trainer – St. Paul, MN**",Job Title
4,05b865e93e8e46579075562865973d3b,388,Our business purpose is to restore health and ...,About Company


Distribution of values by type of sections

In [16]:
job_training_df.section_label.value_counts()

Job Responsibilities/Summary    1453
Job Skills/Requirements         1012
Other                            506
About Company                    425
Benefits                         291
EOE/Diversity                    163
Job Title                         35
Name: section_label, dtype: int64

In [17]:
job_training_df.section_label.value_counts()/job_training_df.shape[0]

Job Responsibilities/Summary    0.374003
Job Skills/Requirements         0.260489
Other                           0.130245
About Company                   0.109395
Benefits                        0.074903
EOE/Diversity                   0.041956
Job Title                       0.009009
Name: section_label, dtype: float64

Previous output shows that there are `seven classes` classes, which are unbalaced. Now, check if there are missing values.

In [18]:
# check for null values
job_training_df.isnull().sum()

job_id           0
segment_index    0
segment          0
section_label    0
dtype: int64

Previous output unveils that there aren't missing values

---

# Section 1. Feature engineering

## Create syntactial features

Data are augmented with features related to the syntactical characteristics of job description sentences.

1. **Word count of sentences**: total number of words in sentences
2. **Character count of sentences**: total number of characters in sentences
3. **Average word density of sentences**: average length of words in sentences
4. **Upper case count in sentences**: total number of upper count words in sentences
5. **Title word count in sentences**: total number of title case words in sentences
6. **Noun count**: total number of nouns in sentences
7. **Verb count**: total number of verbs in sentences
8. **Adjective count**: total number of adjectives in sentences
9. **Pronouns count**: total number of pronouns in sentences

In [19]:
%%time
syn_features_df = pd.DataFrame()
for idx, row in job_training_df.iterrows():
  syn_features_df = pd.concat([syn_features_df, create_syntactical_features(row['segment'], idx)])

CPU times: user 35.3 s, sys: 204 ms, total: 35.5 s
Wall time: 36.3 s


In [20]:
syn_features_df.head()

,noun_count,verb_count,adj_count,adv_count,pron_count,chart_count,word_count,word_density,upper_case_word_count,title_word_count
0,6,3,1,2,0,93,17,5.166667,0,1
1,10,2,4,0,0,192,24,7.680000,0,1
2,3,1,1,0,0,63,10,5.727273,0,1
3,9,0,1,0,0,33,6,4.714286,1,4
4,14,3,0,0,0,184,29,6.133333,2,1


Concate the new features with the job description data

In [21]:
job_training_df = pd.concat([job_training_df, syn_features_df.reindex(job_training_df.index)], axis=1)

In [22]:
job_training_df.head()

,job_id,segment_index,segment,section_label,noun_count,verb_count,adj_count,adv_count,pron_count,chart_count,word_count,word_density,upper_case_word_count,title_word_count
0,05b865e93e8e46579075562865973d3b,0,Abbott is a global healthcare leader that help...,About Company,6,3,1,2,0,93,17,5.166667,0,1
1,05b865e93e8e46579075562865973d3b,94,Our portfolio of life-changing technologies sp...,About Company,10,2,4,0,0,192,24,7.680000,0,1
2,05b865e93e8e46579075562865973d3b,287,"Our 109,000 colleagues\nserve people in more t...",About Company,3,1,1,0,0,63,10,5.727273,0,1
3,05b865e93e8e46579075562865973d3b,353,"**Tissue Trainer – St. Paul, MN**",Job Title,9,0,1,0,0,33,6,4.714286,1,4
4,05b865e93e8e46579075562865973d3b,388,Our business purpose is to restore health and ...,About Company,14,3,0,0,0,184,29,6.133333,2,1


## Pre-process text

Pre-process job description sentences to be used  by the machine learning algorithms. In this sense, sentences are converted to a list of `lower case tokens`, `removing` in this process the `stops words`, `punctuations`, and `digits`.

In [23]:
processed_segs = preprocess_segments(job_training_df['segment'])

In [24]:
# Let's explore tokens of the first segment
processed_segs[0]

['abbott',
 'global',
 'healthcare',
 'leader',
 'helps',
 'people',
 'live',
 'fully',
 'stages',
 'life']

## Encode labels

The target variable `section_label` contains categorical data, which need to be converted to numbers before usign them to train machine learning algorithms.

In [25]:
labels = list(job_training_df['section_label'].unique())
label_values = list(job_training_df['section_label'].values)
encoded_labels = encode_labels(labels, label_values, models_dir)

## Split dataset

Split the dataset into train and test, holding 10% for testing

In [34]:
features, labels = pd.concat([pd.Series(processed_segs, name='segment_tokens'), job_training_df['segment_index'],
                              job_training_df.iloc[:,4:]], axis=1), encoded_labels
features_train, features_test, y_train, y_test = train_test_split(features, labels, random_state=random_state,
                                                                  test_size=0.10, stratify=labels)

In [35]:
features_train.head(3)

,segment_tokens,segment_index,noun_count,verb_count,adj_count,adv_count,pron_count,chart_count,word_count,word_density,upper_case_word_count,title_word_count
1818,"[must, uk, national, evidence, required, eithe...",6134,12,4,5,1,0,215,38,5.512821,2,6
1191,[],1946,1,0,0,0,0,1,1,0.500000,0,0
553,"[assumes, individual, responsibilities, may, a...",2466,10,3,3,1,0,213,27,7.607143,0,1


In [36]:
y_train[:3]

array([4, 3, 3])

Check if the proportion of target classes has been preserved

In [39]:
labels_train = decode_labels(y_train, os.path.join(models_dir, 'encoder_classes.npy'))
labels_train[:3]

['Job Skills/Requirements',
 'Job Responsibilities/Summary',
 'Job Responsibilities/Summary']

In [41]:
unique, counts = np.unique(labels_train, return_counts=True)
dict(zip(unique, counts/len(labels_train)))

{'About Company': 0.10926773455377574,
 'Benefits': 0.07494279176201372,
 'EOE/Diversity': 0.04204805491990847,
 'Job Responsibilities/Summary': 0.37414187643020597,
 'Job Skills/Requirements': 0.26058352402745993,
 'Job Title': 0.008867276887871853,
 'Other': 0.1301487414187643}

## Scale non-textual features

Scale numerical features (i.e., the syntactical created before and the `segment_index`) so values are closer to word vectors that are generated in the next steps

In [43]:
scaled_features_array = scale_features(np.array(features_train.iloc[:,1:]), models_dir)
scaled_features_df = pd.DataFrame(scaled_features_array, columns=list(features_train.columns)[1:])

In [44]:
scaled_features_df.head()

,segment_index,noun_count,verb_count,adj_count,adv_count,pron_count,chart_count,word_count,word_density,upper_case_word_count,title_word_count
0,2.002221,0.462094,0.652813,1.353352,0.972226,-0.176252,0.763056,1.165647,-0.378565,0.815929,0.932006
1,-0.139451,-0.774615,-0.926041,-0.819969,-0.482161,-0.176252,-0.935479,-0.931153,-3.394293,-0.199011,-0.706562
2,0.126468,0.237237,0.258099,0.484024,0.972226,-0.176252,0.747182,0.542274,0.881386,-0.199011,-0.433468
3,-0.922378,-0.549759,-0.136614,0.484024,0.972226,-0.176252,-0.189393,-0.251110,0.387212,-0.199011,0.112722
4,-0.840045,0.237237,-0.136614,0.484024,0.972226,-0.176252,0.612252,0.372263,1.021474,-0.199011,-0.433468


## Create embeddings using doc2vec

Job descriptions are used to create word embeddings of sentences. `Doc2vec` is employed to learn the vector the representation of job description sentences. The vector length and learning approach were decided based on the results of experiments with different combinations of sizes (`50`, `100`, `200`, `300`, `400`, `500`) and the doc2vec approaches `distributed memory` and `distributed bag of words` implemented by the [Gensim](https://radimrehurek.com/gensim/models/doc2vec.html) library.

In [47]:
%%time
doc2vec_model_fn = os.path.join(models_dir, 'doc2vec.model')
if OVERWRITE_DOC2VEC or not os.path.isfile(doc2vec_model_fn):
  hyper_params = {
      'vector_size': 500,
      'alpha': 0.025,
      'min_count': 5,
      'dm': 0,
      'epochs': 100
  }
  print('Preparing sentences for training...')
  train_doc2vec = [TaggedDocument((d), tags=[str(i)]) for i, d in enumerate(list(features_train['segment_tokens']))]
  print('-'*10)
  print('Training embeddings...')
  doc2vec_model = Doc2Vec(**hyper_params)
  doc2vec_model.build_vocab(train_doc2vec)
  doc2vec_model.train(train_doc2vec, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)
  print('-'*10)
  print('Doc2vec model successfully created!')
  print('-'*10)
  print('Saving model...')
  print('-'*10)
  doc2vec_model.save(doc2vec_model_fn)
else:
  print('Loading doc2vec model...')
  doc2vec_model = Doc2Vec.load(doc2vec_model_fn)

Loading doc2vec model...
CPU times: user 22.7 ms, sys: 17.6 ms, total: 40.3 ms
Wall time: 1.15 s


## Create doc vector features

Pre-trained embeddings are used to generate vectors for sentences

In [48]:
%%time
print('Creating vectors...')
if doc2vec_model:
  train_vectors =  [doc2vec_model.infer_vector(seg_tokens) for seg_tokens in list(features_train.segment_tokens)]
print('-'*10)

Creating vectors...
----------
CPU times: user 16.6 s, sys: 8.31 ms, total: 16.6 s
Wall time: 18.2 s


In [49]:
# let's explore an example of a generated doc vector
train_vectors[0]

array([ 0.16402999, -0.01964702,  0.20051296,  0.20283973,  0.03550222,
        0.08361363, -0.235325  , -0.23937277,  0.32608014, -0.48931804,
       -0.11774838, -0.25637344,  0.10906851, -0.01124543,  0.01026615,
       -0.13404307,  0.1081857 , -0.24574743,  0.16619827,  0.02237521,
        0.05840909,  0.10361686, -0.08074299,  0.20511943, -0.04438891,
        0.02758475,  0.0713376 , -0.28029543, -0.04899712,  0.20525955,
       -0.18326746,  0.21180005, -0.17451048, -0.18823455, -0.20913902,
        0.05721293, -0.05154701, -0.16588707, -0.22950402,  0.07594979,
       -0.12759277,  0.2942918 ,  0.01004684, -0.36563894,  0.20420848,
        0.02497183, -0.01402901,  0.28487536,  0.14857888, -0.08346214,
       -0.18482128, -0.24610896,  0.01259339, -0.02565778, -0.0600877 ,
       -0.17692742, -0.07215507, -0.1539186 ,  0.25099796,  0.18971187,
        0.00399008, -0.09891257,  0.01344062, -0.02538432,  0.0082141 ,
        0.15530206,  0.07076819,  0.03492387, -0.10746431, -0.28

---

# Section 2. Model building

## Build ML model

Here the machine learning classifier is built. The decision on the algorithms to be used was based on algorithms that have been reported to perform well on unbalanced, small, and textual datasets (e.g., [Text Classification with Extremely Small Datasets](https://towardsdatascience.com/text-classification-with-extremely-small-datasets-333d322caee2)). Therefore, we try the following algorithms: `Support Vector Machine` and `Logistic Regression`.

In bulding the model we proceed as following:

* Grid search is used for training algorithms with different combinations of hyperparamets;
* Best models for each algortihm are pre-selected for testing;
* Testing performance of best models are compared and the model that shows the best results is selected.

Models are trained using the cross-validation approach. The performance metric to be optimized is weighted `F1` because it provides an adequate compromise between acceptable coverage and the correct identification of impact sentences.

### Train classifiers

In [50]:
# concatenate word vectors with numerical features
X_train = np.concatenate((train_vectors, scaled_features_df), axis=1)

In [51]:
def get_lr_parameters():
    """
    Define hyper-parameters of
    Logistic Regression.

    Inspired by
    https://notebook.community/tpin3694/tpin3694.github.io/machine-learning/.ipynb_checkpoints/hyperparameter_tuning_using_grid_search-checkpoint
    """
    param_grid = {
        'penalty': ['l2', None],
        'C': np.logspace(0, 4, 10)
    }
    return param_grid

In [52]:
def get_svm_parameters():
    """
    Define hyper-parameters of
    Support Vector Machine.

    Inspired by
    https://www.vebuso.com/2020/03/svm-hyperparameter-tuning-using-gridsearchcv
    """

    param_grid = {
        'C': [0.01, 0.1, 1, 10, 100],
        'gamma': [1, 0.1, 0.01, 0.001, 'scale'],
        'kernel': ['linear', 'rbf', 'poly']
    }
    return param_grid

In [53]:
def filter_params(model_params, hyperparams):
  filtered_params = {}
  for param, value in model_params.items():
    if param in hyperparams:
      filtered_params[param] = value
  return filtered_params

In [68]:
%%time
output_training_file_path = os.path.join(output_dir, 'output_training.json')
if not os.path.isfile(output_training_file_path) or OVERWRITE_TRAINING:
  kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
  metric = {
      'name': 'f1',
      'obj': make_scorer(f1_score, average='weighted')
  }
  training_outputs = []

  for algorithm in ['LR', 'SVM']:
    print(f'Training {algorithm} models with different combination of hyperparameters...')
    if algorithm == 'SVM':
      # class_weight='balanced' is used to indicate that classes are not balanced in the dataset
      classifier = SVC(random_state=random_state, class_weight='balanced')
      hyperparams = get_svm_parameters()
    else:
      classifier = LogisticRegression(random_state=random_state,
                                      class_weight='balanced', max_iter=200,
                                      solver='lbfgs')
      hyperparams = get_lr_parameters()
    # create grid search
    clf = GridSearchCV(classifier, param_grid=hyperparams, cv=kfold,
                       scoring=metric['obj'], verbose=2, error_score='raise',
                       n_jobs=1)
    # do grid search
    best_model = clf.fit(X_train, y_train)
    score = best_model.best_score_
    print(f'Finished training {algorithm} models, {metric["name"]} score of best model: {score}')
    print(f'Hyperparameters of the best {algorithm} model are:')
    print(filter_params(best_model.best_estimator_.get_params(), hyperparams))
    print(f'Saving the best model {algorithm} model...')
    model_name = f'{algorithm.lower()}_model.joblib'
    model_file_path = os.path.join(models_dir, model_name)
    model_dict = dict(model=best_model)
    joblib.dump(model_dict, model_file_path)
    training_outputs.append(
      {
          'algorithm': algorithm,
          'hyperparms': filter_params(best_model.best_estimator_.get_params(), hyperparams),
          'metric': metric['name'],
          'score': score,
          'model_file_path': model_file_path
      }
    )
    print('-'*10)
  # save training outputs
  print('Saving outputs...')
  with open(output_training_file_path, 'w') as f:
      json.dump(training_outputs, f, indent=4)
  print('-'*10)
else:
  print('Skipping, training outputs already exist...')

Training LR models with different combination of hyperparameters...
Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END ..................................C=1.0, penalty=l2; total time=   1.9s
[CV] END ..................................C=1.0, penalty=l2; total time=   2.0s
[CV] END ..................................C=1.0, penalty=l2; total time=   1.9s
[CV] END ..................................C=1.0, penalty=l2; total time=   2.4s
[CV] END ..................................C=1.0, penalty=l2; total time=   3.1s
[CV] END ................................C=1.0, penalty=None; total time=   4.0s
[CV] END ................................C=1.0, penalty=None; total time=   2.2s
[CV] END ................................C=1.0, penalty=None; total time=   2.2s
[CV] END ................................C=1.0, penalty=None; total time=   2.3s
[CV] END ................................C=1.0, penalty=None; total time=   2.3s
[CV] END ...................C=2.7825594022071245, penalty=l2

### Evaluate best classifiers

#### Prepare test set

Scale numerical features

In [69]:
scaled_features_array = scale_features(np.array(features_test.iloc[:,1:]), models_dir)
scaled_features_df = pd.DataFrame(scaled_features_array, columns=list(features_train.columns)[1:])

Generate doc vectors of segments in the test split

In [70]:
# load doc2vec model
doc2vec_model_file_path = os.path.join(models_dir, 'doc2vec.model')
doc2vec_model = Doc2Vec.load(doc2vec_model_file_path)
test_vectors =  [doc2vec_model.infer_vector(seg_tokens) for seg_tokens in list(features_test.segment_tokens)]

Concatenate vectors with numerical features

In [71]:
X_test = np.concatenate((test_vectors, scaled_features_df), axis=1)

#### **Model**: Best Logistic Regression classifier

Load model

In [72]:
output_training_file_path = os.path.join(output_dir, 'output_training.json')
with open(output_training_file_path, 'r') as f:
    training_outputs = json.load(f)
for training in training_outputs:
  if training['algorithm'] == 'LR':
    model_file_path = training['model_file_path']
best_lr_model = joblib.load(model_file_path)['model']

Make predictions

In [73]:
preds = best_lr_model.predict(X_test)

Evaluate model performance

In [74]:
enconder_file_path = os.path.join(models_dir, 'encoder_classes.npy')
class_nums = list(range(0,7))
print(classification_report(y_test, preds, target_names=decode_labels(class_nums, enconder_file_path)))

                              precision    recall  f1-score   support

               About Company       0.49      0.67      0.57        43
                    Benefits       0.53      0.55      0.54        29
               EOE/Diversity       0.62      0.94      0.75        16
Job Responsibilities/Summary       0.84      0.69      0.76       145
     Job Skills/Requirements       0.82      0.74      0.78       101
                   Job Title       0.38      0.75      0.50         4
                       Other       0.60      0.69      0.64        51

                    accuracy                           0.70       389
                   macro avg       0.61      0.72      0.65       389
                weighted avg       0.73      0.70      0.71       389



#### **Model**: Best Support Vector Machine classifier

Load model

In [75]:
output_training_file_path = os.path.join(output_dir, 'output_training.json')
with open(output_training_file_path, 'r') as f:
    training_outputs = json.load(f)
for training in training_outputs:
  if training['algorithm'] == 'SVM':
    model_file_path = training['model_file_path']
best_svm_model = joblib.load(model_file_path)['model']

Make predictions

In [76]:
preds = best_svm_model.predict(X_test)

Evaluate model performance

In [77]:
enconder_file_path = os.path.join(models_dir, 'encoder_classes.npy')
class_nums = list(range(0,7))
print(classification_report(y_test, preds, target_names=decode_labels(class_nums, enconder_file_path)))

                              precision    recall  f1-score   support

               About Company       0.50      0.67      0.57        43
                    Benefits       0.73      0.55      0.63        29
               EOE/Diversity       0.82      0.88      0.85        16
Job Responsibilities/Summary       0.82      0.79      0.80       145
     Job Skills/Requirements       0.87      0.80      0.84       101
                   Job Title       1.00      1.00      1.00         4
                       Other       0.73      0.80      0.77        51

                    accuracy                           0.77       389
                   macro avg       0.78      0.78      0.78       389
                weighted avg       0.78      0.77      0.77       389



### Select best-perfomed model

> From results above it is clear that the `support vector machine` model shows better a better performance with an `accuracy` of `0.77` and a similar weighted average `f1` in comparison with its `logistic regression` counterpart, which shows an `accuracy` of `0.70` and a weighted average `f1` of `0.71`. Therefore, the `support vector machine` model will be used for predictions.

---

# Section 3. Use case

### Predict section of segments

Solution is checked by predicting the section of a given sentence (and sentence index) taken from the dataset `jobs_test`.

In [78]:
segment = 'The company began more than 100 years ago in Tulsa and has successfully diversified into a variety of industries, businesses and geographies. .'
segment_idx = 341

In [79]:
segments_df = pd.DataFrame({'segment': [segment], 'segment_index': [segment_idx]})
segments_df.head()

,segment,segment_index
0,The company began more than 100 years ago in T...,341


Create syntactical features for the segment

In [80]:
num_features_df = create_syntactical_features(segment)
num_features_df.head()

,noun_count,verb_count,adj_count,adv_count,pron_count,chart_count,word_count,word_density,upper_case_word_count,title_word_count
0,7,3,1,2,0,143,23,5.958333,0,2


In [81]:
num_features_df = pd.concat([segments_df['segment_index'], num_features_df], axis=1)
num_features_df.head()

,segment_index,noun_count,verb_count,adj_count,adv_count,pron_count,chart_count,word_count,word_density,upper_case_word_count,title_word_count
0,341,7,3,1,2,0,143,23,5.958333,0,2


Scale numerical features

In [82]:
scaled_features_array = scale_features(np.array(num_features_df.iloc[:,:]), models_dir)
scaled_num_features_df = pd.DataFrame(scaled_features_array, columns=list(features_train.columns)[1:])
scaled_num_features_df.head()

,segment_index,noun_count,verb_count,adj_count,adv_count,pron_count,chart_count,word_count,word_density,upper_case_word_count,title_word_count
0,-0.96022,-0.100047,0.258099,-0.385304,2.426613,-0.176252,0.191587,0.315593,-0.110543,-0.199011,-0.160373


Pre-process segment

In [83]:
processed_segs = preprocess_segments(segments_df['segment'])
processed_segs[0]

['company',
 'began',
 'years',
 'ago',
 'tulsa',
 'successfully',
 'diversified',
 'variety',
 'industries',
 'businesses',
 'geographies']

Generate doc vector for segment

In [84]:
# load doc2vec model
doc2vec_model_file_path = os.path.join(models_dir, 'doc2vec.model')
doc2vec_model = Doc2Vec.load(doc2vec_model_file_path)
segment_vector =  [doc2vec_model.infer_vector(seg_tokens) for seg_tokens in processed_segs]

Concatenate vector with numerical features

In [85]:
seg_features = np.concatenate((segment_vector, scaled_num_features_df), axis=1)

Load model

In [86]:
output_training_file_path = os.path.join(output_dir, 'output_training.json')
with open(output_training_file_path, 'r') as f:
    training_outputs = json.load(f)
for training in training_outputs:
  if training['algorithm'] == 'SVM':
    model_file_path = training['model_file_path']
svm_model = joblib.load(model_file_path)['model']

Make prediction

In [87]:
pred = svm_model.predict(seg_features)

Output prediction result

In [88]:
class_names = decode_labels(list(range(0,7)), os.path.join(models_dir, 'encoder_classes.npy'))
for class_num, class_name in zip(list(range(0,7)), class_names):
  if class_num == pred[0]:
    print('Prediction result')
    print('-'*10)
    print(f'Segment: {segment}')
    print(f'Predicted section: {class_name}')
    print('-'*10)
    break

Prediction result
----------
Segment: The company began more than 100 years ago in Tulsa and has successfully diversified into a variety of industries, businesses and geographies. .
Predicted section: About Company
----------


---